# PyQGIS basics
## 1. Introduction
This Jupyter Notebook will introduce you to the basics of using PyQGIS. You can run all commands from the QGIS Python Console, but here we use the notebook to provide instructions.

Before starting this tutorial you need to download the files from the [main course page on GIS OpenCourseWare](https://courses.gisopencourseware.org/course/section.php?id=587).

## 2. Loading vector layers
We will first use a shapefile called *river_regions_mdb.shp* which contains polygons for each river region in the Murray-Darling river catchment. It is located in a subdirectory *shp*. In order to make sure that PyQGIS is able to find this file without too much diffculty, you should first open the QGIS project file *PyQGIS_Tutorial.qgz*. The project file is empy at the start but we will use Python code to add the shapefile to it.

After opening the project we can run the code below.

In [ ]:
layers = iface.layerTreeView()
print(layers.currentLayer()) # returns None if project is empty

Since there are no layers in this project yet, the return value **None** will be printed. What we tried here is to get access to the current map layer in the Layers treeview window (*Layers* panel) which is by default located in the lower left part of the QGIS window. This introduces the important `iface` class, which provides access to methods to interact with the QGIS interface. You can use it to perform task that you would normally execute yourself by entering commands or using mouse clicks in QGIS. The project does not yet contain layers, which is why the attempt to print the current layer in the treeview results in the word **None** appearing as a result.

Since we opened the project file we can get also get access to the current project that is opened in QGIS. It has an attribute called `homePath`, which tells us the directory where the project’s qgz file is stored.

In [ ]:
current_project = QgsProject.instance()
home_path = current_project.homePath()
print(home_path)

Once we know the homepath we can get the full path to the shapefile that we want to import. For this we use the `join` function from Python’s native os library. It stitches together the names of directories and a file into a full path. Note that this library works across operating systems so it is best practice to create folder names this way rather than to provide them as strings yourself.

Here we create the path to `river_regions_mdb.shp`:

In [ ]:
fpath_shp = os.path.join(
   current_project.homePath(), "shp", "river_regions_mdb", "river_regions_mdb.shp"
)

Print `fpath_shp` to see the result of the path to the shapefile.

Once the file location has been determined, we can use the `addVectorLayer` method to import the shapefile. The function takes the filename as the first argument, then the layer name as the second and finally the provider, which is the code that will read the file. For more information see the [following post on StackExchange](https://gis.stackexchange.com/q/383425).

In [ ]:
vlayer = iface.addVectorLayer(
   vectorLayerPath=fpath_shp,
   baseName="river regions",
   providerKey="ogr",
)

You will note that we store the return value of `addVectorLayer` in a variable that we call `vlayer`. Now that the layer has been added to the *Layers treeview*, its current layer should be equal to `vlayer`, which can be checked by printing the equality check:

In [ ]:
current_layer = layers.currentLayer()
print(current_layer == vlayer)

## 3. Applying filters
We now also have access to the fields in the attribute table. In a `for` loop we can print their names.

In [ ]:
for field in vlayer.fields():
   print(field.name())

We can also loop over the features and print some of their field values.

In [ ]:
for feature in vlayer.getFeatures():
   print(feature["rivregname"], feature["Shape__Are"])

Let’s create a copy of the layer and set a filter so that only the river regions of which the *Shape_Are* field is larger than 1e11 m2. The clone method of the original layer returns an identical copy. We then use the `setName` method to change the name of the new layer and the `setSubsetString` method to create the filter. Once we are happy with the new layer, we can add it to the project using the `addMapLayer` method of the `current_project` object that we had stored earlier.

In [ ]:
new_vlayer = vlayer.clone()
new_vlayer.setName("large catchments")
new_vlayer.setSubsetString('"Shape__Are" > 1e11')
current_project.addMapLayer(new_vlayer)

We can now print the information about the filtered river regions and present the areas in ha:

In [ ]:
for feature in new_vlayer.getFeatures():
   print(
      "{name} has an area of {area: .2f} ha".format(
         name=feature["rivregname"], area=feature["Shape__are"] / 10000
   )
)

## 4. Styling the vector layer
Now that we have added the layer with only a subset of the river regions, we should change the polygons colours in order to make them stand out against the layer with all the river regions. This means that we first have to create a new QGIS colour, red in this case, that we can assign to the colour property of the symbol of the new layer’s renderer. Note the chained sequence of method calls. The last line ensures that the symbol in the *Layers treeview* is also updated to reflect the new appearance of the layer.

In [ ]:
from qgis.PyQt.QtGui import QColor
new_color = QColor("red")
new_vlayer.renderer().symbol().setColor(new_color)
new_vlayer.triggerRepaint()

## 5. Loading raster layers
Now we’re going to load a raster layer. The raster provided with the course materials is an SRTM 1-Arc Second DEM. It is located in a subdirectory *tif* and is named *SRTM30.tif*.

Loading a raster is very similar to loading a vector layer. We first create the string with the path to the file:

In [ ]:
fpath_tif = os.path.join(current_project.homePath(), "tif", "SRTM30.tif")

We can load the layer using the `addRasterLayer` method from the `iface` class. Below we also check for the validity of the raster. You can use the same code for vector layers.

In [ ]:
rlayer = iface.addRasterLayer(url=fpath_tif, layerName="SRTM30",
   providerKey="gdal")

if rlayer.isValid():
   print("This is a valid raster layer!")
else:
   print("This raster layer is invalid!")

We can print some properties of the loaded raster:

In [ ]:
print("Width: {}px".format(rlayer.width()))
print("Height: {}px".format(rlayer.height()))
print("Extent: {}".format(rlayer.extent().toString()))

Notice that we used `rlayer.extent().toString()` instead of just `rlayer.extent()`. If we don’t use `toString()`, the output would look like this: Extent: `<qgis._core.QgsRectangle object at 0x0000028CDA596E58>`. This indicates that `rlayer.extent()` returns a `QgsRectangle` object, but it doesn’t provide the actual extent of the layer.

We can also print statistics of the raster values:

In [ ]:
stats = rlayer.dataProvider().bandStatistics(1)
minimum = stats.minimumValue
maximum = stats.maximumValue
print("Min value: {} m".format(minimum))
print("Max value: {} m".format(maximum))

In our case we have a singleband raster. Therefore we use `bandStatistics(1)`. More info of what you can do with this method can be found in the [documentation](https://qgis.org/pyqgis/master/core/QgsRasterBandStats.html). Try a few.

## 6. Styling the raster layer
Styling rasters with PyQGIS is a bit more complex than styling of vector layers.
We first need to create a colour ramp shader:

In [ ]:
from qgis.core import QgsColorRampShader

rshader = QgsColorRampShader()

Next, we need to set the type of colour ramp: 
* Interpolated: stretches the colours across a range of values 
* Discrete: gives all values in a range the same colour
* Exact: gives each unique pixel value a unique colour.

In our case we have to use *Interpolated*, because our raster represents continuous elevations:

In [ ]:
rshader.setColorRampType(QgsColorRampShader.Interpolated)

Now we need to define the colours between which it has to interpolate. This is defined as a list of `ColorRampItems`, consisting of a value and a colour. We’ll use a ramp that goes from green for low values to yellow for high values.

In [ ]:
lst = [
   QgsColorRampShader.ColorRampItem(minimum, QColor(0, 255, 0)),
   QgsColorRampShader.ColorRampItem(maximum, QColor(255, 255, 0)),
]
rshader.setColorRampItemList(lst)

Note that you can add a list of more value/colour combinations to create a ramp that interpolates the colours.

Next, we can assign the defined colour ramp to the `QgsRasterShader` class that will be used to apply the ramp to our raster layer:

In [ ]:
from qgis.core import QgsRasterShader

shader = QgsRasterShader()
shader.setRasterShaderFunction(rshader)

The last step is to apply the defined symbology to the raster layer. We’ll do that with the *Singleband pseudocolor* renderer:

In [ ]:
from qgis.core import QgsSingleBandPseudoColorRenderer

renderer = QgsSingleBandPseudoColorRenderer(rlayer.dataProvider(), 1, shader)
rlayer.setRenderer(renderer)
rlayer.triggerRepaint()

If the contrast is a bit low, you can replace the minimum and maximum values.

## 7. Running Processing Tools with PyQGIS
With PyQGIS we can also run processing tools. The easiest way is to run a tool and get its code from the history of the processing toolbox.

Let’s reproject the DEM. 
* Go to main menu and choose **Raster | Projections | Warp (reproject)…**. 
* Fill in the dialog. Use for nodata a value of -9999 and choose EPSG:3577 as target projection (use the button on the right of the edit field to get acces to the CRS selection window). Leave the rest as default.

After running the tool, remove the output layer. We’ll now create the same result using a PyQGIS script.

Go to the *Processing Toolbox* panel and click on the *History* icon (the little clock icon). There look for the entry with the warp tool that you’ve just used and click on the little black arrow to expand the selected entry. The first item there, with the Python icon, will show the code, where all dialog entries are put in a Python dictionary. Click with the right mouse button on the item and select **Copy as Python Command**. Paste the code below. Reformat the code as in the example below to improve its
readability.

In [ ]:
processing.run(
   "gdal:warpreproject", 
   {
      'INPUT':'Z:/Python_tutorials/PyQGIS/tif/SRTM30.tif',
      'SOURCE_CRS':None,
      'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:3577'),
      'RESAMPLING':0,
      'NODATA':-9999,
      'TARGET_RESOLUTION':None,
      'CREATION_OPTIONS':None,
      'DATA_TYPE':0,
      'TARGET_EXTENT':None,
      'TARGET_EXTENT_CRS':None,
      'MULTITHREADING':False,
      'EXTRA':'',
      'OUTPUT': 'Z:/Python_tutorials/PyQGIS/tif/DEM_reprojected.tif'
   }
)

As you see there’s no error, but also the result is not added to the project. In this way you can use processing scripts as intermediate processing steps in a workflow that have no need of loading intermediate layers.

However, here we are interested in the result. To show the result, you need to replace the `run` method with `runAndLoadResults`. Do this below and verify the result.

## 8. Assignment
Let’s put the elements together in a little assignment.
First we start clean by removing all layers from the *Layers* panel:

In [ ]:
current_project.removeAllMapLayers()
iface.mapCanvas().refresh()


Note that the `QgsProject.instance()` method (which is in our case in the `current_project`
variable, see earlier) can be used to also add and remove individual layers.

Now, perform the following steps:

1 Load the reprojected DEM into the *Layers* panel
2 Use the *GDAL Slope* processing tool with PyQGIS to calculate the slope of the DEM (note that you can also copy the Python code by clicking the ‘Advanced’ button in the lower left corner of the dialog window of the tool).
3 Manipulate the Python dictionary in such a way that slopes are calculated in percentage and that edges are calculated.
4 Add a nice colour ramp to the slope raster
5 Save the result as python file that can be run from the QGIS Python Console.